In [1]:
import pandas as pd
import numpy as np

In [117]:
# ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
# nrats = pd.read_sql('select "RAT.NUM_ATIVIDADE" from "tbl_rat"', 'sqlite:///gdo.db')
# rats[
#     rats['RAT.NUM_ATIVIDADE'] == '2020-006129150-001'
# ]['MUNICIPIO']

In [136]:
nrats_on_db = funcs.get_nrats_on_db()

In [124]:
'2020-006129150-001' in nrats

True

In [121]:
ratv[
    ratv['NUM_ATIVIDADE'] == '2020-006129150-001 '
]

,NUM_ATIVIDADE,NUM_SEQ_RECURSO,NUM_PLACA,NUM_PREFIXO


In [110]:
nrats[
    nrats['RAT.NUM_ATIVIDADE'] == '2020-006129150-001'
]

,RAT.NUM_ATIVIDADE
40846,2020-006129150-001


In [111]:
%%time
import vars

######################### CRIA df_rat
df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

df_rat = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) |
    (df_rat['NAT.CODIGO'] == 'Y04012')  
]

df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

######################### BUSCA DADOS DE VIATURAS E ACRESCENTA EM df_rat
df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
df_ratv = df_ratv['NUM_ATIVIDADE'].value_counts()
df_ratv.rename('VIATURAS', inplace=True)
df_rat = df_rat.join(df_ratv, how='left')
del df_ratv
df_rat['VIATURAS'].fillna(0, inplace=True)
df_rat['VIATURAS'] = df_rat['VIATURAS'].astype('uint8')

######################### BUSCA DADOS DE EFETIVO E ACRESCENTA EM df_rat
df_rate = pd.read_sql_table('tbl_rat_efetivo', 'sqlite:///gdo.db')
df_rate = df_rate['NUM_ATIVIDADE'].value_counts()
df_rate.rename('EFETIVO', inplace=True)
df_rat = df_rat.join(df_rate, how='left')
del df_rate
df_rat['EFETIVO'].fillna(0, inplace=True)
df_rat['EFETIVO'] = df_rat['EFETIVO'].astype('uint8')

######################### BUSCA DADOS DE PRODUTIVIDADE E ACRESCENTA EM df_rat
df_ratp = pd.read_sql_table('tbl_rat_produtividade', 'sqlite:///gdo.db')
df_ratp_efet = df_ratp[
    df_ratp['DESCRICAO'].isin(vars.itens_efet)
].drop_duplicates('RAT.NUM_ATIVIDADE').set_index('RAT.NUM_ATIVIDADE')['QUANTIDADE'].map(lambda qtd:1).rename('EFETIVIDADE_PARCIAL')
df_rat = df_rat.join(df_ratp_efet, how='left')
del df_ratp_efet
df_rat['EFETIVIDADE_PARCIAL'].fillna(0, inplace=True)
df_rat['EFETIVIDADE_PARCIAL'] = df_rat['EFETIVIDADE_PARCIAL'].astype('int8')

df_itens_ee = df_ratp[
    df_ratp['DESCRICAO'].isin([
        'Qde de veiculos fiscalizados',
        'Qde de pessoas abordadas',
        'Qde de locais fiscalizados',
        'Qde de pessoas que sopraram o etilometro'
    ])
]
df_itens_ee = pd.pivot_table(df_itens_ee, columns='DESCRICAO', index='RAT.NUM_ATIVIDADE')
df_itens_ee.columns = df_itens_ee.columns.droplevel()
df_rat = df_rat.join(df_itens_ee, how='left')
for i in [
    'Qde de veiculos fiscalizados',
    'Qde de pessoas abordadas',
    'Qde de locais fiscalizados',
    'Qde de pessoas que sopraram o etilometro'
]:
    df_rat[i].fillna(0, inplace=True)
    df_rat[i] = df_rat[i].astype('int16')
del df_ratp

cond71 = (
    # Y07001 - OPERACAO DE BATIDA POLICIAL
    (
        (df_rat['NAT.CODIGO'] == 'Y07001') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2)
    )
)
cond73 = (
    # Y07003 - OPERACAO DE INCURSAO EM ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07003') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )
)
cond74 = (
    # Y07004 - OPERACAO DE CERCO / BLOQUEIO / INTERCEPTACAO
    (
        (
            (df_rat['NAT.CODIGO'] == 'Y07004') &            
            (df_rat['EFETIVO'] >= 3) &
            (df_rat['VIATURAS'] >= 2)
        ) &
        (
            (df_rat['Qde de pessoas abordadas'] >= 1) |
            (df_rat['Qde de veiculos fiscalizados'] >= 1)
        )
    )
)
cond75 = (
    # Y07005 - OPERACAO DE OCUPACAO DE PTOS DE ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07005') &
        (df_rat['TEMPO_INT'] >= 60) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )    
)
cond412 = (
    # Y04012 - OPERACAO LEI SECA
    (
        (df_rat['NAT.CODIGO'] == 'Y04012') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 3) |
        (df_rat['Qde de pessoas que sopraram o etilometro'] >= 3) |
        (df_rat['Qde de veiculos fiscalizados'] >= 3)
    )
)

cond_ee = [
    cond71 |
    cond73 |
    cond74 |
    cond75 |
    cond412
]

df_rat['EFICIENCIA_E_EFICACIA'] = np.select(cond_ee,[1],default=0)
df_rat['EFETIVIDADE'] = np.select(
    [(df_rat['EFETIVIDADE_PARCIAL'] == 1) & (df_rat['EFICIENCIA_E_EFICACIA'] == 1)],
    [1],
    default=0
)

df_rat.head(3)

CPU times: user 2.13 s, sys: 24 ms, total: 2.16 s
Wall time: 2.15 s


,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0
2018-051709149-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/11/2018,20/11/2018,05:25,20/11/2018,07:15,TRANSEUNTE,METALURGICA / SIDERURGICA,,...,142 CIA,1,2,0,0,0,0,0,0,0
2018-052156427-001,Y07001,OPERACAO DE BATIDA POLICIAL,22/11/2018,22/11/2018,20:20,22/11/2018,20:45,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,142 CIA,1,2,0,0,0,0,0,0,0


<h2 style='color:#009'>Código para gerar relatório de LEI SECA, por Cias, por ano, por mês</h2>

In [112]:
df_rat_ols = df_rat[        
    (~df_rat['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
    & (df_rat['NAT.CODIGO'] == 'Y04012')
]
df_rat_ols.loc[:,'CIA_OLS'] = np.select([
    df_rat_ols.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('240')
], [
    '240 CIA'
    ], default = df_rat_ols.loc[:,'CIA'])

ols_by_cia = df_rat_ols[
    df_rat_ols['ANO'] >= 2020
].groupby(['ANO','MES','CIA_OLS']).sum()['EFETIVIDADE'].unstack(['MES']).fillna(0)

ols_by_cia.loc[:,'ACUM'] = ols_by_cia.sum(1)
ols_by_cia.loc[(2020, '23 BPM'),:] = ols_by_cia.sum()

# df_rat_ols

ols_by_cia = ols_by_cia.astype('int16')
ols_by_cia.columns =  pd.MultiIndex.from_product([['OLS'], ols_by_cia.columns])
ols_by_cia

/home/wendel/.asdf/installs/python/anaconda3-2019.07/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/wendel/.asdf/installs/python/anaconda3-2019.07/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


OLS              
               1  2  3  4 ACUM
ANO  CIA_OLS                  
2020 139 CIA   1  0  1  0    2
     142 CIA   0  1  0  0    1
     240 CIA   0  1  2  3    6
     51 CIA    0  3  1  0    4
     53 CIA    2  0  0  0    2
     23 BPM    3  5  4  3   15

<h2 style='color:#009'>Código para gerar relatório de RQV, por setores, por ano, por mês</h2>

In [113]:
df_rqv_23 = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (df_rat['ANO'] == 2020)
            
].groupby(['ANO','MES','CIA']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]


df_rqv_23 = df_rqv_23.unstack(level='MES')
df_rqv_23.columns = df_rqv_23.columns.set_levels(df_rqv_23.columns.levels[1].astype('int32'), level=1)
df_rqv_23.index = df_rqv_23.index.set_levels(df_rqv_23.index.levels[0].astype('int32'), level=0)
df_rqv_23.loc[:,('EFICIENCIA_E_EFICACIA','ACUM')] = df_rqv_23['EFICIENCIA_E_EFICACIA'].sum(1)
df_rqv_23.loc[:,('EFETIVIDADE','ACUM')] = df_rqv_23['EFETIVIDADE'].sum(1)
df_rqv_23 = df_rqv_23[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]
df_rqv_23.loc[(2020,'23 BPM'),:] = df_rqv_23.sum()
df_rqv_23.fillna(0, inplace=True)
df_rqv_23 = df_rqv_23.astype('int32')
df_rqv_23

EFICIENCIA_E_EFICACIA                      EFETIVIDADE            \
MES                              1    2    3    4  ACUM           1    2    3   
ANO  CIA                                                                        
2020 139 CIA                   214  235  208   99   756          44   40   37   
     142 CIA                   416  368  337  147  1268         142  142  138   
     51 CIA                    251  153  126   79   609          92   59   63   
     53 CIA                    221  175  188  101   685          54   30   27   
     23 BPM                   1102  931  859  426  3318         332  271  265   

                         
MES             4  ACUM  
ANO  CIA                 
2020 139 CIA   24   145  
     142 CIA   61   483  
     51 CIA    31   245  
     53 CIA    18   129  
     23 BPM   134  1002

In [7]:
writer = pd.ExcelWriter('rqv_e_ols.xlsx', engine='xlsxwriter')
df_rqv_23.to_excel(writer, sheet_name='RQV')
df_ols_resumo.to_excel(writer, sheet_name='OLS')
writer.save()

In [91]:
rqv_cinds = pd.read_excel('files/PLANILHA RQV ATE 20ABR20.xlsx', sheet_name='DADOS')

In [92]:
import funcs
rqv_cinds = funcs.filter_23(rqv_cinds)

In [93]:
rqv_cinds = rqv_cinds[
    ( ~ rqv_cinds['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ rqv_cinds['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ rqv_cinds['NOM_UNID_RESPONSAVEL'].str.contains('CPE'))
]

In [35]:
rqv_cinds.columns

Index(['RAT.NUM_ATIVIDADE', 'DTA_INICIO', 'TEMPO', 'NATUREZA', 'RPM_NOVA',
       'CPE', 'CIA PM IND PE', 'NOM_UNIDADE_AREA', 'NOM_UNID_RESPONSAVEL',
       'MUNICIPIO', 'MES_RAT', 'UEOP_2018', 'EFETIVO', 'VIATURA',
       'EFICIENCIA - Y07001', 'TOTAL', 'PESSOAS', 'VEICULOS', 'LOCAIS',
       'EFICÁCIA', 'PESSOAS_PRESAS', 'ARMAS', 'DROGAS', 'VEICULO_REC_AP',
       'EFETIVIDADE', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'],
      dtype='object')

In [114]:
rqv_pandas = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]'))
    & ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('IND PE'))
    & ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('BPE'))
    & ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPE'))
    & (df_rat['ANO'] == 2020)
            
]
rqv_efet_pandas = rqv_pandas[
    (rqv_pandas['ANO'] == 2020)
    & rqv_pandas['EFETIVIDADE'] == 1
].sort_index()

In [95]:
rqv_efet_cinds = rqv_cinds[
    rqv_cinds['EFETIVIDADE'] == 1.
].sort_values(by='RAT.NUM_ATIVIDADE')

In [96]:
rqv_efet_cinds.set_index('RAT.NUM_ATIVIDADE', inplace=True)

In [115]:
rqv_efet_pandas.index.difference(rqv_efet_cinds.index)

Index([], dtype='object', name='RAT.NUM_ATIVIDADE')

In [116]:
rqv_efet_cinds.index.difference(rqv_efet_pandas.index)

Index(['2020-006129150-001', '2020-017273099-001', '2020-017298810-001',
       '2020-017892613-001'],
      dtype='object', name='RAT.NUM_ATIVIDADE')

In [98]:
rqv_efet_cinds.loc['2020-006129150-001']

DTA_INICIO                  2020-02-05 00:00:00
TEMPO                                  08:22:00
NATUREZA                                 Y07003
RPM_NOVA                                7ª RISP
CPE                                      OUTROS
CIA PM IND PE                            OUTROS
NOM_UNIDADE_AREA        142 CIA PM/23 BPM/7 RPM
NOM_UNID_RESPONSAVEL    240 CIA TM/23 BPM/7 RPM
MUNICIPIO                           DIVINOPOLIS
MES_RAT                                     fev
UEOP_2018                                23 BPM
EFETIVO                                       3
VIATURA                                       1
EFICIENCIA - Y07001                           1
TOTAL                                         1
PESSOAS                                      31
VEICULOS                                      5
LOCAIS                                        4
EFICÁCIA                                      1
PESSOAS_PRESAS                                2
ARMAS                                   

In [41]:
df_rat.loc['2020-006129150-001']

NAT.CODIGO                                                                             Y07003
NAT.DESCRICAO                               OPERACAO DE INCURSAO EM ZONA QUENTE DE CRIMINA...
DTA_HRA_INCLUSAO                                                                   05/02/2020
DTA_INICIO                                                                         05/02/2020
HRA_INICIO                                                                              08:08
DTA_TERMINO                                                                        05/02/2020
HRA_TERMINO                                                                             16:30
DES_ALVO_EVENTO                                                                    TRANSEUNTE
DES_LUGAR                                                               VIA DE ACESSO PUBLICA
NOM_OPERACAO                                                                                 
COD_UNIDADE_SERVICO                                         

In [45]:
nrats_on_db = funcs.get_nrats_on_db()

In [47]:
ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')

In [54]:
ratv[
    ratv['NUM_ATIVIDADE'] == '2020-006129150-001'
]

,NUM_ATIVIDADE,NUM_SEQ_RECURSO,NUM_PLACA,NUM_PREFIXO


In [46]:
'2020-006129150-001' in nrats_on_db

True

In [64]:
rqv_efet_cinds.loc[['2020-006129150-001', '2020-017273099-001', '2020-017298810-001',
       '2020-017892613-001']]

,DTA_INICIO,TEMPO,NATUREZA,RPM_NOVA,CPE,CIA PM IND PE,NOM_UNIDADE_AREA,NOM_UNID_RESPONSAVEL,MUNICIPIO,MES_RAT,...,LOCAIS,EFICÁCIA,PESSOAS_PRESAS,ARMAS,DROGAS,VEICULO_REC_AP,EFETIVIDADE,Unnamed: 25,Unnamed: 26,Unnamed: 27
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2020-006129150-001,2020-02-05 00:00:00,08:22:00,Y07003,7ª RISP,OUTROS,OUTROS,142 CIA PM/23 BPM/7 RPM,240 CIA TM/23 BPM/7 RPM,DIVINOPOLIS,fev,...,4.0,1.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
2020-017273099-001,2020-04-10 00:00:00,01:10:00,Y07001,7ª RISP,OUTROS,OUTROS,142 CIA PM/23 BPM/7 RPM,240 CIA TM/23 BPM/7 RPM,DIVINOPOLIS,abr,...,2.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
2020-017298810-001,2020-04-10 00:00:00,00:40:00,Y07001,7ª RISP,OUTROS,OUTROS,142 CIA PM/23 BPM/7 RPM,240 CIA TM/23 BPM/7 RPM,DIVINOPOLIS,abr,...,3.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
2020-017892613-001,2020-04-14 00:00:00,03:43:00,Y07003,7ª RISP,OUTROS,OUTROS,139 CIA PM/23 BPM/7 RPM,240 CIA TM/23 BPM/7 RPM,DIVINOPOLIS,abr,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN


In [65]:
rat_all = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
rat_all.set_index('RAT.NUM_ATIVIDADE', inplace=True)

In [67]:
ratv_db = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
ratv_db.set_index('NUM_ATIVIDADE', inplace=True)

In [75]:
ratv_db.loc['2020-006129150-001']

KeyError: '2020-006129150-001'

In [66]:
rat_all.loc['2020-006129150-001']

NAT.CODIGO                                                         Y07003
NAT.DESCRICAO           OPERACAO DE INCURSAO EM ZONA QUENTE DE CRIMINA...
DTA_HRA_INCLUSAO                                               05/02/2020
DTA_INICIO                                                     05/02/2020
HRA_INICIO                                                          08:08
DTA_TERMINO                                                    05/02/2020
HRA_TERMINO                                                         16:30
DES_ALVO_EVENTO                                                TRANSEUNTE
DES_LUGAR                                           VIA DE ACESSO PUBLICA
NOM_OPERACAO                                                             
COD_UNIDADE_SERVICO                                                 M7342
NOM_UNID_RESPONSAVEL                              240 CIA TM/23 BPM/7 RPM
TIPO_LOGRADOURO                                                       RUA
LOGRADOURO                            

In [111]:
a = pd.read_sql('SELECT * FROM "tbl_rat_efetivo" WHERE "NUM_ATIVIDADE" == "2020-000214900-001"', 'sqlite:///gdo.db')

In [199]:
files = funcs.get_files_names()
files

{'rat': ['REDS_RAT_20190101_20190110.csv',
  'REDS_RAT_20190111_20190120.csv',
  'REDS_RAT_20190121_20190131.csv',
  'REDS_RAT_20190201_20190210.csv',
  'REDS_RAT_20190211_20190220.csv',
  'REDS_RAT_20190221_20190228.csv',
  'REDS_RAT_20190301_20190310.csv',
  'REDS_RAT_20190311_20190320.csv',
  'REDS_RAT_20190321_20190331.csv',
  'REDS_RAT_20190401_20190410.csv',
  'REDS_RAT_20190411_20190420.csv',
  'REDS_RAT_20190421_20190430.csv',
  'REDS_RAT_20190501_20190510.csv',
  'REDS_RAT_20190511_20190520.csv',
  'REDS_RAT_20190521_20190531.csv',
  'REDS_RAT_20190601_20190610.csv',
  'REDS_RAT_20190611_20190620.csv',
  'REDS_RAT_20190621_20190630.csv',
  'REDS_RAT_20190701_20190710.csv',
  'REDS_RAT_20190711_20190720.csv',
  'REDS_RAT_20190721_20190731.csv',
  'REDS_RAT_20190801_20190810.csv',
  'REDS_RAT_20190811_20190820.csv',
  'REDS_RAT_20190821_20190831.csv',
  'REDS_RAT_20190901_20190910.csv',
  'REDS_RAT_20190911_20190920.csv',
  'REDS_RAT_20190921_20190930.csv',
  'REDS_RAT_20191001_

In [137]:
fevv1[
    fevv1.iloc[:,0].isin(nrats_on_db)
]

,NUM_ATIVIDADE,NUM_SEQ_RECURSO,NUM_PLACA,NUM_PREFIXO


In [130]:
fevv1[
    fevv1['NUM_ATIVIDADE'] == '2020-006129150-001 '
]

,NUM_ATIVIDADE,NUM_SEQ_RECURSO,NUM_PLACA,NUM_PREFIXO
31102,2020-006129150-001,1,QUR4316,28861.0


In [129]:
import csv

fev1 = pd.read_csv('files/RAT/REDS_RAT_20200201_20200210.csv', sep='|', quoting=csv.QUOTE_NONE)

In [126]:
import csv

fevv1 = pd.read_csv('files/RAT/REDS_RAT_VIATURAS_20200201_20200210.csv', sep='|', quoting=csv.QUOTE_NONE)

In [128]:
fevv1[
    fevv1['NUM_ATIVIDADE'] == '2020-006129150-001 '
]

,NUM_ATIVIDADE,NUM_SEQ_RECURSO,NUM_PLACA,NUM_PREFIXO
31102,2020-006129150-001,1,QUR4316,28861.0


In [55]:
import csv
fevv1 = pd.read_csv('files/RAT/REDS_RAT_VIATURAS_20200201_20200210.csv', sep='|', quoting=csv.QUOTE_NONE)